In [27]:
import os, glob, time
import numpy as np
import cv2
from PIL import Image
Image.MAX_IMAGE_PIXELS = 2560*2160*10    # 画像サイズが大きいときはここの数値を大きくする

In [28]:
import k3d

In [29]:
import k3d_util as util

In [30]:
# データの格納場所
data_path = './tutorial_spheroid.npz'

# 対物レンズ倍率
util.MAGNIFICATION = 40

# 撮影時のビニング
util.BINNING = 1

# Z間隔
util.Z_SLICE_LENGTH = 2.0

# 画像の画素数を減らすための画像縮小率
util.REDUCE_RATIO = 4

# Zスタックの枚数を減らすためのZステップ間隔
util.Z_STEP = 1

In [31]:
# 3D画像データの読み込み関数の定義
def load_data(file_list):
    file_list.sort()
    img = Image.open(file_list[0])
    buf = np.zeros((len(file_list), img.size[1]//util.REDUCE_RATIO, img.size[0]//util.REDUCE_RATIO), dtype=np.float32)
    for z, i in enumerate(file_list[::util.Z_STEP]):
        img = Image.open(i)
        img = np.asarray(img, dtype=np.float32)
        # y軸の正方向が画面奥になるため(通常2D画像はy軸の正方向は下)、::-1で行を反転する
        img = cv2.resize(img[::-1, :], (buf.shape[2], buf.shape[1]), interpolation=cv2.INTER_AREA)
        buf[z, :, :] = img
    return buf

In [32]:
#チャネルごとに3D画像データを読み込む
#data1 = load_data(glob.glob(os.path.join(data_path, 'W0040*C1.tif')), binning=binning)
#data2 = load_data(glob.glob(os.path.join(data_path, 'W0040*C2.tif')), binning=binning)
#np.savez_compressed(data_path, data1=data1, data2=data2)
saved_data = np.load(data_path)
data_list = [saved_data['data1'], saved_data['data2']]

In [33]:
# 3D画像データの寸法を計算する
util.init_bounds(data_list[0])

In [38]:
# volume_listに画像データ(複数チャネル)
ch1 = k3d.mip(data_list[0], 
                   util.COLOR_MAP_LIST[0], 
                   color_range=[100, 1100],
                   samples=512, gradient_step=0, 
                   bounds=(0, util.cx, 0, util.cy, 0, util.cz)
)
ch2 = k3d.mip(data_list[1], 
                   util.COLOR_MAP_LIST[1], 
                   color_range=[100, 500],
                   samples=512, gradient_step=0, 
                   bounds=(0, util.cx, 0, util.cy, 0, util.cz)
)

In [39]:
# 3Dビューアの表示エリアを生成
plot = k3d.plot(background_color=0x000000, grid_visible=False, lighting=0, height=540)
plot += ch2
plot += ch1

In [40]:
plot.display()


Output()

In [42]:
plot.camera_fov = 20